In [121]:
import numpy as np
import pandas as pd

data = pd.read_csv("dataset/dataset_train.csv")

In [122]:
courses = ['Astronomy', 'Herbology', 'Divination', 'Muggle Studies', 
            'Ancient Runes', 'History of Magic', 'Transfiguration']

features = data[courses]
features = np.array(features.fillna(features.median()))
features = features.T
x_test = features

houses = sorted(list(set(data["Hogwarts House"])))

m = features.shape[1]
n = features.shape[0]
k = len(houses)

print(features.shape)
print(houses)
print(m,n, k)


(7, 1600)
['Gryffindor', 'Hufflepuff', 'Ravenclaw', 'Slytherin']
1600 7 4


In [123]:
mean = np.sum(features, axis=1, keepdims=True) / m

variance = np.sum((features - mean) ** 2, axis=1, keepdims=True) / m
std = np.sqrt(variance)

x_train = (features - mean) / std

print(mean.shape)
print(std.shape)
print(mean)
print(std)

(7, 1)
(7, 1)
[[  44.2069772 ]
 [   1.18903438]
 [   3.18974313]
 [-228.8462294 ]
 [ 495.05169614]
 [   3.00112491]
 [1030.42440995]]
[[515.82921406]
 [  5.17451398]
 [  4.10928909]
 [481.68392363]
 [105.18574767]
 [  4.37049142]
 [ 43.69638037]]


In [124]:
def predict(w, x, b):
    z = np.dot(w.T, x) + b
    return 1 / (1 + np.exp(-z))

def train(x, y, alpha=0.1, iter=10000000, limit=0.0000001):
    
    m = x.shape[1]
    n = x.shape[0]

    w = np.zeros((n, 1))
    b = 0

    for i in range(iter):
        p = predict(w, x, b)
        loss = -(1/m) * np.sum(y*np.log(p) + (1-y)*np.log(1-p))
        
        if (i % 1000 == 0):
            print(f"Iteration: {i} | Loss: {loss}")

        step_w = alpha * (1/m) * np.dot(x, (p - y).T)
        step_b = alpha * (1/m) * np.sum(p - y)

        w = w - step_w
        b = b - step_b

        if (np.all(np.abs(step_w)) <= limit and np.abs(step_b) <= limit):
            print("Converge")
            break

    print("Didn't converge")
    return w, b

In [125]:
w_all = np.zeros((n,k))
b_all = np.zeros((k,1))

for i, h in enumerate(houses):
    print(f"Train for {h} house")
    y = np.where(data["Hogwarts House"] == h, 1, 0).astype(np.uint8)
    y_train = y.reshape(1, m)


    w, b = train(x_train, y_train)

    w_denorm = w / std
    b_denorm = b - np.sum(w * mean / std)
    print(w)
    print(b)
    print("---------")

    w_all[:, i:i + 1] = w_denorm
    b_all[i, 0] = b_denorm

Train for Gryffindor house
Iteration: 0 | Loss: 0.6931471805599452
Iteration: 1000 | Loss: 0.05017321837648034
Iteration: 2000 | Loss: 0.0481850924282886
Iteration: 3000 | Loss: 0.04750567113658663
Iteration: 4000 | Loss: 0.047136617341936624
Iteration: 5000 | Loss: 0.046908502349019714
Iteration: 6000 | Loss: 0.04676116604839823
Iteration: 7000 | Loss: 0.046664073094424253
Iteration: 8000 | Loss: 0.04659925677085795
Iteration: 9000 | Loss: 0.04655550864222947
Iteration: 10000 | Loss: 0.04652566003898576
Iteration: 11000 | Loss: 0.046505067127420724
Iteration: 12000 | Loss: 0.046490695180440404
Iteration: 13000 | Loss: 0.04648054539355112
Iteration: 14000 | Loss: 0.04647329062969511
Iteration: 15000 | Loss: 0.046468042130740565
Iteration: 16000 | Loss: 0.04646419926653932
Iteration: 17000 | Loss: 0.04646135223056141
Iteration: 18000 | Loss: 0.04645921863840501
Iteration: 19000 | Loss: 0.04645760193297889
Iteration: 20000 | Loss: 0.04645636389064299
Iteration: 21000 | Loss: 0.0464554063

KeyboardInterrupt: 

In [ ]:
y_predict = predict(w_all, x_test, b_all)
answer = np.argmax(y_predict, axis=0)
print(answer)
print(answer.shape)


In [ ]:
validate = data["Hogwarts House"]

output = []
for i in answer:
    output.append(houses[i])

count = 0
for i,j in zip(validate, output):
    if (i == j): count += 1

print(count / 1600 * 100)
